# Neural network 
1. Ustawienie losowych wag i biasów.
2. Przepuszenie otrzymanych wyników przez funkcje aktywacji aby dodać nielinowość.
3. Obliczenie błędu przewidywań wyników.
4. Uruchomienie propagacji wstecznej aby poprawić skuteczność przewidywanych wyników.
5. Trening modelu.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
def init_weight_bias():
    weights = []
    biases = []
    
    for _ in range(3):
        weights.append(np.random.rand(2, 2) - 0.5)
        biases.append(np.random.rand(1, 2) - 0.5)

    return weights, biases

Wyczytałem, że dobrze jest odjąć 0.5 od losowo wygenerowanej liczby bo stwarza to zakrec który zawiera liczby dodatnie jak i ujemne oraz wartości unikalne które sprzyjają lepszemu uczeniu się modelowi i nie powodują uczenia się w ten sam sposób na tych samych wagach i odchyleniach. Jest to optymalizacja problemu symmetry breaking. Odjęcie od bias wartości 0.5 pomaga w uniknięciu stronniczości w uczeniu maszynowym, jest to podobno standardowa inicjalizacja.

In [3]:
weights, biases = init_weight_bias()

print("Losowe wagi: ", weights)
print("Losowe biasy: ", biases)

Losowe wagi:  [array([[-0.3972919 , -0.41351816],
       [-0.24413513, -0.24488088]]), array([[0.44098236, 0.16399595],
       [0.24643083, 0.36768975]]), array([[-0.1578589 ,  0.25876884],
       [ 0.47409877, -0.43352638]])]
Losowe biasy:  [array([[0.22138048, 0.32468206]]), array([[-0.16939359, -0.00586325]]), array([[-0.41070221, -0.10139723]])]


![alt text](images/warstwa.png "Layer")

In [4]:
# Abstrakcyjna wartwa bazowa
def layer(input_X, output_Y):
    input_X = None
    output_Y = None

    return input_X, output_Y

![alt text](images/sigmoid.png "funkcja aktywacji")

Zdecydowałem się na funkcje aktywacji sigmoid ponieważ wyczytałem, że jest idealna do klasyfkikacji binarnej 0,1 

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

![alt text](images/softmax.png "softmax")

In [6]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

Miałem nadzieję, że funkcja softmax dedykowana do używania na wyjściowej warstwie poprawi wyniki mojego modelu.

![alt text](images/propagacja_do_przodu.png "Propagacja do przodu")

![alt text](images/suma_wazona.png "Suma wazona")

In [7]:
def forward_propagation(x, weights, biases):
    # Pierwszy layer
    z1 = np.dot(x, weights[0]) + biases[0]
    result_activation_function = sigmoid(z1)
    # Drugi layer
    z2 = np.dot(result_activation_function, weights[1]) + biases[1]
    result_activation_function2 = sigmoid(z2)

    # Output layer
    z3 = np.dot(result_activation_function2, weights[2]) + biases[2]
    # output= sigmoid(z3)
    # Użyje jednak funkcji softmax na końcu sieci
    output = softmax(z3)


    return result_activation_function, result_activation_function2, output

In [8]:
def forward_propagation(x, W1, b1, W2, b2, W3, b3):
    # Wejscie pierwszej warstwy
    z1 = np.dot(x, W1) + b1
    # Funkcja aktywacji
    a1 = sigmoid(z1)

    # Wejście do drugiej warstwy (tzw. ukrytej - "hidden")
    z2 = np.dot(a1, W2) + b2 
    a2 = sigmoid(z2)

    # Wejście do warstwy wyjściowej (tzw. output layer)
    z3 = np.dot(a2, W3) + b3
    output_layer = sigmoid(z3)

    return a1, a2, output_layer

Doszlismy do końca sieci więc należy policzyć straty aby poprawić wyniki przewidywań. 

In [9]:
def mse_loss(y_true, output_predict):
    y_true = y_true.reshape(-1, 1) # Reshape danych do tablic 2D
    output_predict = output_predict.reshape(-1, 1) # Reshape danych do tablic 2D
    output_error = mean_squared_error(y_true, output_predict)
    return output_error

Zdecydowałem się zastasować średni błęd kwadratowy ponieważ jest to dość popularna metoda i używałem tego wcześniej do sprawdzenia straty przy przewidywań modelu Linii Regresyjnej.

In [10]:
# Cost function: Binary cross-entropy loss
def compute_cost(y, y_hat):
    m = y.shape[0]                            # Number of samples
    y_hat = np.clip(y_hat, 1e-10, 1 - 1e-10)  # Avoid log(0) errors by clipping predictions

    # Vectorized implementation of binary cross-entropy loss
    cost = -(1 / m) * np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    return cost

Entropia krzyżowa niestety idzie w parze z funkcją softmax więc musiałem zrezygnować z MSE

![alt text](images/backpropagation.png "Backpropagation")

In [11]:
# def backward_propagation(x, output_error, weights, biases, learning_rate):

#     weights = np.array(weights)
#     biases = np.array(biases)

#     input_error = np.dot(output_error, weights.T)
#     weights_error = np.dot(x.T, output_error)

#     print(weights.shape)
#     print(weights_error.shape)
#     print(output_error.shape)

#     # Aktualizacja wag i baisow
#     for i in range(len(weights) - 1):
#         weights -= learning_rate * weights_error
#         biases -= learning_rate * output_error 

#     return weights, biases


Nie mogłem tego zrobić na swój własny sposób, bo mi się kształty nie zgadzały.

In [12]:
def backward_propagation(x, y, a1, a2, output, W3, W2):
    # m = y.shape[0]
    m = x.shape[1]
    error = output - y

    # Obliczanie gradientow propagacji wstecznej dla wartwy wyjscia
    dW3 = (1 / m) * np.dot(a2.T, error) / m
    db3 = (1 / m) * np.sum(error, axis=0, keepdims=True)

    # Obliczanie gradientu propagacji wstecznej dla pierwszej wartwy
    dz2 = np.dot(error, W3.T) * (a2 * (1 - a2)) 
    dW2 = (1 / m) * np.dot(a1.T, dz2) / m
    db2 = (1 / m) * np.sum(dz2, axis=0, keepdims=True) / m

    # Obliczanie gradientu propagacji wstecznej dla pierwszej wartwy
    dz1 = np.dot(dz2, W2.T) * (a1 * (1 - a1))
    dW1 = (1 / m) * np.dot(x.T, dz1) / m
    db1 = (1 / m) * np.sum(dz1, axis=0, keepdims=True) / m

    # # Aktualizacja wag i biasów
    # weights[2] -= learning_rate * dW3
    # biases[2] -= learning_rate * db3
    # weights[1] -= learning_rate * dW2
    # biases[1] -= learning_rate * db2
    # weights[0] -= learning_rate * dW1
    # biases[0] -= learning_rate * db1

    return dW1, db1, dW2, db2, dW3, db3

In [13]:
def update_weights_bias(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3

    return W1, b1, W2, b2, W3, b3  

Nie potrafiłem sobie poradzić z implementacją propagacji wstecznej, dlatego użyłem kodu znalezionego na necie.

# Test sieci neuronowej

In [14]:
# x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 
# y_predict = np.array([[0], [1], [1], [0]])  

# learning_rate = 0.01
# epochs = 10000

# for epoch in range(epochs):
#     # Propagacja w przód
#     layer1, layer2, output = forward_propagation(x_train, weights, biases)

#     # Obliczenie kosztu straty po dojściu na koniec sieci
#     # loss = mse_loss(y_predict, output)
#     loss = compute_cost(y_predict, output)

#     # Propagacja wsteczna
#     update_weights, update_biases = backward_propagation(x_train, y_predict, layer1, layer2, output, weights, biases, learning_rate)

#     if epoch % 1000 == 0:
#         print(f"Iteracje/Epoki {epoch}, Strata : {loss:.4f}")

In [15]:
# x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
# y_train = np.array([[0],[1],[1],[0]])

# fetures_1 = x_train.shape[1]
# neurons_1 = x_train.shape[1]

# W1_1, b1_1 = init_weight_bias(fetures_1, neurons_1)
# W2_2, b2_2 = init_weight_bias(fetures_1, neurons_1)
# W3_3, b3_3 = init_weight_bias(fetures_1, 1)

# x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 
# y_predict = np.array([[0], [1], [1], [0]])  

# learning_rate = 0.1
# epochs = 10000

# for epoch in range(epochs):
#     # Propagacja w przód
#     layer1, layer2, output = forward_propagation(x_train, weights, biases)

#     # Obliczenie kosztu straty po dojściu na koniec sieci
#     # loss = mse_loss(y_predict, output)
#     loss = compute_cost(y_predict, output)

#     # Propagacja wsteczna
#     update_weights, update_biases = backward_propagation(x_train, y_predict, layer1, layer2, output, weights, biases, learning_rate)

#     if epoch % 1000 == 0:
#         print(f"Iteracje/Epoki {epoch}, Strata : {loss:.4f}")

In [16]:
def init_weight_bias(number_of_features, number_neuron_of_layer):
    weights = np.random.rand(number_of_features, number_neuron_of_layer)
    bias = np.zeros((1, number_neuron_of_layer))

    return weights, bias

In [20]:
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[1],[1],[0]])

fetures_1 = x_train.shape[1]
neurons_1 = x_train.shape[1]

W1_1, b1_1 = init_weight_bias(fetures_1, neurons_1)
W2_2, b2_2 = init_weight_bias(fetures_1, neurons_1)
W3_3, b3_3 = init_weight_bias(fetures_1, 1)

x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 
y_predict = np.array([[0], [1], [1], [0]])  

learning_rate = 0.1
epochs = 10000

for epoch in range(epochs):
    # Propagacja w przód
    layer1, layer2, output = forward_propagation(x_train, W1_1, b1_1, W2_2, b2_2, W3_3, b3_3)

    # Obliczenie kosztu straty po dojściu na koniec sieci
    # loss = mse_loss(y_predict, output)
    loss = compute_cost(y_predict, output)

    # Propagacja wsteczna
    dW1_1, db1_1, dW2_2, db2_2, dW3_3, db3_3 = backward_propagation(x_train, y_predict, layer1, layer2, output, W3_3, W2_2)
    
    # Aktualizacja wag i biasow
    W1_1, b1_1, W2_2, b2_2, W3_3, b3_3 = update_weights_bias(W1_1, b1_1, W2_2, b2_2, W3_3, b3_3, dW1_1, db1_1, dW2_2, db2_2, dW3_3, db3_3, learning_rate)

    if epoch % 1000 == 0:
        print(f"Iteracje/Epoki {epoch}, Strata : {loss:.4f}")

Iteracje/Epoki 0, Strata : 0.8092
Iteracje/Epoki 1000, Strata : 0.6916
Iteracje/Epoki 2000, Strata : 0.6891
Iteracje/Epoki 3000, Strata : 0.6811
Iteracje/Epoki 4000, Strata : 0.6450
Iteracje/Epoki 5000, Strata : 0.5499
Iteracje/Epoki 6000, Strata : 0.4648
Iteracje/Epoki 7000, Strata : 0.0781
Iteracje/Epoki 8000, Strata : 0.0255
Iteracje/Epoki 9000, Strata : 0.0143


In [21]:
def predict(W1, b1, W2, b2, W3, b3, test):
    a1, a2, output = forward_propagation(test, W1_1, b1_1, W2_2, b2_2, W3_3, b3_3)
    output_a3 = np.squeeze(output)

    prediction = 1 if output >= 0.5 else 0

    print("Dane:", test.tolist(), "Odpowiedź:", prediction)

In [22]:
test = np.array([[0,0]])
predict(W1_1, b1_1, W2_2, b2_2, W3_3, b3_3, test)
test = np.array([[0,1]])
predict(W1_1, b1_1, W2_2, b2_2, W3_3, b3_3, test)
test = np.array([[1,0]])
predict(W1_1, b1_1, W2_2, b2_2, W3_3, b3_3, test)
test = np.array([[1,1]])
predict(W1_1, b1_1, W2_2, b2_2, W3_3, b3_3, test)

Dane: [[0, 0]] Odpowiedź: 0
Dane: [[0, 1]] Odpowiedź: 1
Dane: [[1, 0]] Odpowiedź: 1
Dane: [[1, 1]] Odpowiedź: 0
